# AutoGen - Financial Investment Asistant
#### References
#### https://microsoft.github.io/autogen/0.2/docs/tutorial/introduction
#### https://microsoft.github.io/autogen/0.2/docs/tutorial/conversation-patterns#sequential-chats
#### https://microsoft.github.io/autogen/0.2/docs/tutorial/tool-use

In [4]:
import os
from IPython.display import Markdown, display
from autogen import ConversableAgent, GroupChat, GroupChatManager

C:\Users\krish\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\krish\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\krish\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [5]:
openai_api_key = "<API_KEY>"
os.environ["OPENAI_API_KEY"] = openai_api_key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm_config = {
    "config_list": [
        {
            "model": "gpt-4o-mini",
            "api_key": os.environ["OPENAI_API_KEY"],
        }
    ],
    "temperature": 0.2,
}

In [8]:
user_proxy = ConversableAgent(
    name="UserProxyAgent",
    system_message=(
        "You represent the user. Clearly state the user's intent to manage investments "
        "and provide financial details when asked."
    ),
    llm_config=llm_config,
    human_input_mode="NEVER",
)

user_proxy.description = "Initiates the investment management request and provides user financial data."

In [10]:
portfolio_analysis_agent = ConversableAgent(
    name="PortfolioAnalysisAgent",
    system_message=(
        "You analyze the user's financial profile and summarize their investment portfolio. "
        "Determine whether the portfolio aligns more with a Growth or Value strategy and explain why."
    ),
    llm_config=llm_config,
    human_input_mode="NEVER",
)

portfolio_analysis_agent.description = (
    "Analyzes salary and current investments to classify the portfolio as Growth or Value."
)

In [12]:
growth_investment_agent = ConversableAgent(
    name="GrowthInvestmentAgent",
    system_message=(
        "You specialize in growth investing. "
        "Provide recommendations for high-growth, higher-risk investment opportunities."
    ),
    llm_config=llm_config,
    human_input_mode="NEVER",
)

growth_investment_agent.description = "Provides aggressive, growth-oriented investment recommendations."

In [14]:
value_investment_agent = ConversableAgent(
    name="ValueInvestmentAgent",
    system_message=(
        "You specialize in value investing. "
        "Recommend stable, long-term, lower-risk investment options."
    ),
    llm_config=llm_config,
    human_input_mode="NEVER",
)

value_investment_agent.description = "Provides conservative, long-term value investment recommendations."

In [16]:
investment_advisor_agent = ConversableAgent(
    name="InvestmentAdvisorAgent",
    system_message=(
        "You are a senior financial advisor. "
        "Combine portfolio analysis and investment recommendations into a single, "
        "clear, personalized financial report for the user."
    ),
    llm_config=llm_config,
    human_input_mode="NEVER",
)

investment_advisor_agent.description = (
    "Compiles all insights into a final personalized financial report."
)

In [18]:
allowed_transitions = {
    user_proxy: [portfolio_analysis_agent],
    portfolio_analysis_agent: [growth_investment_agent, value_investment_agent],
    growth_investment_agent: [investment_advisor_agent],
    value_investment_agent: [investment_advisor_agent],
    investment_advisor_agent: [],
}

In [20]:
group_chat = GroupChat(
    agents=[
        user_proxy,
        portfolio_analysis_agent,
        growth_investment_agent,
        value_investment_agent,
        investment_advisor_agent,
    ],
    messages=[],
    max_round=8,
    send_introductions=True,
    allowed_or_disallowed_speaker_transitions=allowed_transitions,
    speaker_transitions_type="allowed",
)

In [22]:
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config,
)

In [24]:
chat_result = user_proxy.initiate_chat(
    group_chat_manager,
    message=(
        "I want help managing my investments.\n\n"
        "Here are my details:\n"
        "- Current Salary: ₹21 LPA\n"
        "- Fixed Deposits: ₹3 lakhs\n"
        "- SIPs / Mutual Funds: ₹6 lakhs\n"
        "- Real Estate: ₹20 lakhs\n"
        "- Gold: ₹5 lakhs\n"
        "- Crypto: ₹2 lakhs\n\n"
        "Please analyze my portfolio and suggest the best investment strategy."
    ),
    summary_method="reflection_with_llm",
)

UserProxyAgent (to chat_manager):

I want help managing my investments.

Here are my details:
- Current Salary: ₹21 LPA
- Fixed Deposits: ₹3 lakhs
- SIPs / Mutual Funds: ₹6 lakhs
- Real Estate: ₹20 lakhs
- Gold: ₹5 lakhs
- Crypto: ₹2 lakhs

Please analyze my portfolio and suggest the best investment strategy.

--------------------------------------------------------------------------------

Next speaker: PortfolioAnalysisAgent

PortfolioAnalysisAgent (to chat_manager):

Based on the financial details you've provided, let's analyze your investment portfolio:

1. **Current Salary**: ₹21 LPA (Lakhs Per Annum)
2. **Fixed Deposits**: ₹3 lakhs
3. **SIPs / Mutual Funds**: ₹6 lakhs
4. **Real Estate**: ₹20 lakhs
5. **Gold**: ₹5 lakhs
6. **Crypto**: ₹2 lakhs

### Portfolio Breakdown:
- **Fixed Deposits (FDs)**: ₹3 lakhs (Conservative, low-risk)
- **SIPs / Mutual Funds**: ₹6 lakhs (Can be growth-oriented or value-oriented depending on the funds chosen)
- **Real Estate**: ₹20 lakhs (Typically a lo

In [28]:
def display_last_message_markdown(chat_result):
    for msg in reversed(chat_result.chat_history):
        content = msg.get("content")
        if isinstance(content, str) and content.strip():
            display(Markdown(content.strip()))
            return

    print("No displayable message found.")

In [30]:
display_last_message_markdown(chat_result)

### Personalized Financial Report

**Client Overview:**
- **Current Salary**: ₹21 LPA
- **Investment Portfolio**:
  - **Fixed Deposits**: ₹3 lakhs
  - **SIPs / Mutual Funds**: ₹6 lakhs
  - **Real Estate**: ₹20 lakhs
  - **Gold**: ₹5 lakhs
  - **Crypto**: ₹2 lakhs

---

### Portfolio Analysis Summary:
Your portfolio exhibits a strong inclination towards value investments, characterized by significant allocations in real estate and gold, complemented by a conservative approach through fixed deposits. While you have some exposure to growth-oriented assets like SIPs and crypto, the overall strategy leans towards stability and long-term value.

### Investment Recommendations:
1. **Diversify SIPs into Value Mutual Funds**:
   - Allocate a portion of your SIPs into value-oriented mutual funds such as:
     - HDFC Value Fund
     - ICICI Prudential Value Discovery Fund
     - Franklin India Value Fund

2. **Consider Real Estate Investment Trusts (REITs)**:
   - Explore REITs to diversify your real estate exposure without the complexities of direct property management.

3. **Invest in Gold ETFs or Sovereign Gold Bonds**:
   - Transition some of your physical gold investment into Gold ETFs or Sovereign Gold Bonds for better liquidity and potential returns.

4. **Explore Dividend Stocks**:
   - Look for blue-chip companies with a consistent history of dividend payments to provide both capital appreciation and income.

5. **Fixed Income Securities**:
   - Consider investing in Public Provident Fund (PPF) or National Pension Scheme (NPS) for stable returns and tax benefits.

6. **Regular Monitoring and Rebalancing**:
   - Keep an eye on your portfolio and rebalance as necessary to ensure alignment with your financial goals.

---

### Next Steps:
- Implement the suggested changes gradually, ensuring comfort with the risk levels.
- Stay informed about market trends and adjust your investments as needed.

---

Thank you for your engagement! Your proactive approach to managing your investments is commendable. Should you have any further questions or require assistance in the future, please feel free to reach out. Wishing you success in your investment journey! Happy investing!